In [ ]:
import logging

import polars as pl
import rootutils
import seaborn as sns

sns.set_style("whitegrid")
logging.basicConfig(level=logging.INFO)

ROOT = rootutils.setup_root(".", pythonpath=True, cwd=True)


pl.Config.set_fmt_str_lengths(200)
pl.Config.set_tbl_cols(50)
pl.Config.set_tbl_rows(50)


In [27]:
EXP_NAME = "en-06"

In [ ]:
src_filenames = [
    "submission_030.csv",
    "submission_033.csv",
    "submission_035.csv",
    "submission_036.csv",
]

sources_dir = ROOT / "notebooks" / "ensemble" / "sources"
df = pl.DataFrame()
for src_filename in src_filenames:
    src_path = sources_dir / src_filename
    i_df = pl.read_csv(src_path).with_row_index()
    df = pl.concat([df, i_df], how="vertical")
    # てきとうに重みづけ
    if "submission_035" in src_filename:
        df = pl.concat([df, i_df], how="vertical")
        df = pl.concat([df, i_df], how="vertical")
    if "submission_033" in src_filename:
        df = pl.concat([df, i_df], how="vertical")


out_dir = ROOT / "data" / "output" / EXP_NAME
out_dir.mkdir(parents=True, exist_ok=True)

df.group_by("index").agg(
    pl.col("チョコレート").mean(),
    pl.col("ビール").mean(),
    pl.col("ヘアケア").mean(),
    pl.col("米（5㎏以下）").mean(),
).sort("index").drop("index").write_csv(out_dir / "submission.csv")
